### Bài tập tính điểm cộng lần 2: Đơn hình theo phương pháp big-M

- MSSV: 22120222
- Họ và tên: Võ Văn Nam
- Lớp: CQ2022/5

**Nội dung bài tập**

SV tham khảo yêu cầu đề bài cụ thể trong slide 3: Cho hệ ràng buộc gồm m điều kiện (với m = 2, 3 hoặc 4) của bài toán QHTT chính tắc n <= 10 biến cùng hệ số của hàm mục tiêu dạng max/min tất cả các số nhập vào đều là số nguyên có trị tuyệt đối không vượt quá 10.

Yêu cầu: Hãy in ra bảng đơn hình xuất phát theo phương pháp big-M với số biến bổ sung là ít nhất, trong đó chọn: M=1000.

In [10]:
'''
Cho hệ ràng buộc gồm m điều kiện (với m = 2, 3 hoặc 4) của bài toán QHTT chính tắc n <= 10 biến 
cùng hệ số của hàm mục tiêu dạng max/min tất cả các số nhập vào đều là số nguyên có trị tuyệt đối không vượt quá 10.
Yêu cầu: Hãy in ra bảng đơn hình xuất phát theo phương pháp big-M với số biến bổ sung là ít nhất, trong đó chọn: M=1000.

Input: 
1 2 3 // hệ số của hàm mục tiêu
1 -1 2 = 3 (dấu này có thể là >=, <= hoặc = ).
3 4 -2 = 5
Output: 
x4  -1000   3   1   -1  2   1   0
x5  -1000   5   3   4   -1  0   1
        -8000   -4001   -3002   -3   0   0
'''

M = 1000  # Giá trị M của Big-M

def parse_constraint(line, n):
    """
    Phân tích chuỗi dòng ràng buộc.
    Dòng có dạng: a1 a2 ... an quan_hệ b
    Quan hệ có thể là '=', '<=' hoặc '>='.
    Trả về: (list các hệ số của biến gốc, quan hệ, b)
    """
    tokens = line.strip().split()
    coef = list(map(int, tokens[:n]))
    rel = tokens[n]  # token thứ n là quan hệ
    b_val = int(tokens[n+1])
    return coef, rel, b_val

def main():
    # Gán cứng input mẫu
    input_data = [
        "1 2 3",       # Hệ số của hàm mục tiêu
        "1 -1 2 = 3",  # Ràng buộc thứ 1
        "3 4 -2 = 5"   # Ràng buộc thứ 2
    ]
    
    # Xử lý input từ danh sách input_data
    lines = [l for l in input_data if l.strip() != '']
    
    # Dòng đầu tiên: hệ số của hàm mục tiêu (cho biến gốc x1,...,xn)
    obj_coeffs = list(map(int, lines[0].strip().split()))
    n = len(obj_coeffs)
    
    constraints = []
    for line in lines[1:]:
        coef, rel, b_val = parse_constraint(line, n)
        constraints.append({
            'coef': coef,
            'rel': rel,
            'b': b_val
        })
    
    m = len(constraints)
    
    # Danh sách chứa thông tin về các biến bổ sung (extra variables)
    # Mỗi phần tử: {'constr': index của ràng buộc, 'type': loại biến, 'c': hệ số trong hàm mục tiêu}
    extra_vars = []
    # Với mỗi hàng ràng buộc, lưu thông tin về biến cơ sở (nếu có)
    basis = [None] * m
    
    # Danh sách chứa vector hệ số của biến bổ sung cho từng hàng
    row_extra = []
    for i, constr in enumerate(constraints):
        current_index = len(extra_vars)
        # Xử lý theo loại quan hệ:
        if constr['rel'] == '<=':
            extra_vars.append({'constr': i, 'type': 'slack', 'c': 0})
            basis[i] = current_index
        elif constr['rel'] == '=':
            extra_vars.append({'constr': i, 'type': 'artificial', 'c': -M})
            basis[i] = current_index
        elif constr['rel'] == '>=':
            extra_vars.append({'constr': i, 'type': 'surplus', 'c': 0})
            extra_vars.append({'constr': i, 'type': 'artificial', 'c': -M})
            basis[i] = current_index + 1
        else:
            print(f"Quan hệ {constr['rel']} không hợp lệ")
            exit(1)
        
        # Xây dựng vector hệ số cho biến bổ sung của hàng i với độ dài hiện tại = số biến bổ sung hiện có
        row_vec = [0] * len(extra_vars)
        # Với từng biến bổ sung liên quan đến hàng i, thiết lập hệ số:
        #  - 1 đối với slack và artificial, -1 đối với surplus.
        for j in range(len(extra_vars)):
            if extra_vars[j]['constr'] == i:
                if extra_vars[j]['type'] in ['slack', 'artificial']:
                    row_vec[j] = 1
                elif extra_vars[j]['type'] == 'surplus':
                    row_vec[j] = -1
        row_extra.append(row_vec)
    
    total_extra = len(extra_vars)
    
    # Cập nhật lại các hàng trong row_extra sao cho độ dài của vector cho biến bổ sung bằng total_extra
    for i in range(len(row_extra)):
        if len(row_extra[i]) < total_extra:
            row_extra[i].extend([0] * (total_extra - len(row_extra[i])))
    
    # Xây dựng bảng đơn hình: mỗi hàng gồm:
    # [Tên biến cơ sở, c_B, b, hệ số của biến gốc, hệ số của các biến bổ sung]
    tableau_rows = []
    for i, constr in enumerate(constraints):
        idx = basis[i]
        cB = extra_vars[idx]['c'] if idx is not None else 0
        row = [cB, constr['b']] + constr['coef'] + row_extra[i]
        # Đặt tên biến: các biến gốc là x1,...,xn; nên các biến bổ sung được đánh số từ x(n+1) trở đi.
        var_index = n + idx + 1 if idx is not None else None
        tableau_rows.append((f"x{var_index}" if var_index is not None else "", row))
    
    # Tính hàng mục tiêu (hàng -z) theo công thức Big-M
    # Hằng số của hàng mục tiêu: - sum(cB * b) của từng hàng
    const_term = 0
    # Tổng số cột bao gồm biến gốc (n cột) và biến bổ sung (total_extra cột)
    num_cols = n + total_extra
    reduced = [0] * num_cols
    for i, constr in enumerate(constraints):
        cB = tableau_rows[i][1][0]  # c_B của hàng i
        b_val = tableau_rows[i][1][1]
        # const_term += - cB * b_val
        const_term += cB * b_val
        # Lấy ra vector hệ số cho biến gốc và biến bổ sung của hàng i
        row_data = tableau_rows[i][1][2:]
        for j in range(num_cols):
            # reduced[j] += - cB * row_data[j]
            reduced[j] += cB * row_data[j]
    
    # Hiệu chỉnh trừ hệ số của hàm mục tiêu ban đầu cho biến gốc
    for j in range(n):
        reduced[j] -= obj_coeffs[j]
    # Và trừ hệ số của biến bổ sung (0 đối với slack/surplus, -M đối với artificial)
    for j in range(n, num_cols):
        extra_cost = extra_vars[j - n]['c']
        reduced[j] -= extra_cost

    objective_row = [const_term] + reduced

    # In ra bảng đơn hình
    for label, row in tableau_rows:
        row_str = " ".join(str(x) for x in row)
        print(f"{label} {row_str}".strip())
    obj_str = " ".join(str(x) for x in objective_row)
    print(" " + obj_str)

if __name__ == '__main__':
    main()


x4 -1000 3 1 -1 2 1 0
x5 -1000 5 3 4 -2 0 1
 -8000 -4001 -3002 -3 0 0
